In [1]:
import sys
import numpy as np
import pandas as pd
import os 
import gc
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import LabelEncoder
import datetime
import time
import lightgbm as lgb
import xgboost as xgb
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [2]:
path  = '../Data/Train/'
train_sales  = pd.read_csv(path+'train_sales_data.csv')
train_search = pd.read_csv(path+'train_search_data.csv')
train_user   = pd.read_csv(path+'train_user_reply_data.csv')
evaluation_public = pd.read_csv(path+'evaluation_public.csv')
submit_example    = pd.read_csv(path+'submit_example.csv')
data = pd.concat([train_sales, evaluation_public], ignore_index=True)
data = data.merge(train_search, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user, 'left', on=['model', 'regYear', 'regMonth'])
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
look1 = train_sales.drop_duplicates('model').set_index('model')
data['bodyType'] = data['model'].map(train_sales.drop_duplicates('model').set_index('model')['bodyType'])
for i in ['bodyType', 'model']:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(data[i].nunique()))))
data['mt'] = (data['regYear'] - 2016) * 12 + data['regMonth']
data['model_adcode'] = data['adcode'] + data['model']

In [3]:
def get_stat_feature(df_):   
    df = df_.copy()
    stat_feat = []
    df['model_adcode'] = df['adcode'] + df['model']
    df['model_adcode_mt'] = df['model_adcode'] * 100 + df['mt']
    for col in tqdm(['label','popularity']):
        # shift
        for i in [1,2,3,4,5,6,7,8,9,10,11,12]:
            stat_feat.append('shift_model_adcode_mt_{}_{}'.format(col,i))
            df['model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'] + i
            df_last = df[~df[col].isnull()].set_index('model_adcode_mt_{}_{}'.format(col,i))
            df['shift_model_adcode_mt_{}_{}'.format(col,i)] = df['model_adcode_mt'].map(df_last[col])
    return df,stat_feat

def score(data, pred='pred_label', label='label', group='model'):
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    data_agg = data.groupby('model').agg({
        pred:  list,
        label: [list, 'mean']
    }).reset_index()
    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)

def get_model_type(train_x,train_y,valid_x,valid_y,m_type='lgb'):   
    if m_type == 'lgb':
        model = lgb.LGBMRegressor(
                                num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='mse',
                                max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=2019,
                                n_estimators=2000, subsample=0.9, colsample_bytree=0.7,
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              categorical_feature=cate_feat, 
              early_stopping_rounds=100, verbose=100)      
    elif m_type == 'xgb':
        model = xgb.XGBRegressor(
                                max_depth=5 , learning_rate=0.05, n_estimators=2000, 
                                objective='reg:gamma', tree_method = 'hist',subsample=0.9, 
                                colsample_bytree=0.7, min_child_samples=5,eval_metric = 'rmse' 
                                )
        model.fit(train_x, train_y, 
              eval_set=[(train_x, train_y),(valid_x, valid_y)], 
              early_stopping_rounds=100, verbose=100)   
    return model

def get_train_model(df_, m, m_type='lgb'):
    df = df_.copy()
    # 数据集划分
    st = 13
    all_idx   = (df['mt'].between(st , m-1))
    train_idx = (df['mt'].between(st , m-5))
    valid_idx = (df['mt'].between(m-4, m-4))
    test_idx  = (df['mt'].between(m  , m  ))
    print('all_idx  :',st ,m-1)
    print('train_idx:',st ,m-5)
    print('valid_idx:',m-4,m-4)
    print('test_idx :',m  ,m  )  
    # 最终确认
    train_x = df[train_idx][features]
    train_y = df[train_idx]['label']
    valid_x = df[valid_idx][features]
    valid_y = df[valid_idx]['label']   
    # get model
    model = get_model_type(train_x,train_y,valid_x,valid_y,m_type)  
    # offline
    df['pred_label'] = model.predict(df[features])
    best_score = score(df[valid_idx]) 
    # online
    if m_type == 'lgb':
        model.n_estimators = model.best_iteration_ + 100
        model.fit(df[all_idx][features], df[all_idx]['label'], categorical_feature=cate_feat)
    elif m_type == 'xgb':
        model.n_estimators = model.best_iteration + 100
        model.fit(df[all_idx][features], df[all_idx]['label'])
    df['forecastVolum'] = model.predict(df[features]) 
    print('valid mean:',df[valid_idx]['pred_label'].mean())
    print('true  mean:',df[valid_idx]['label'].mean())
    print('test  mean:',df[test_idx]['forecastVolum'].mean())
    # 阶段结果
    sub = df[test_idx][['id']]
    sub['forecastVolum'] = df[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)  
    return sub,df[valid_idx]['pred_label']



In [4]:
def trend_factor(data):
    '''计算趋势因子'''
    for col in ['adcode', 'model', 'model_adcode']:
        temp_df = pd.DataFrame(columns=[col, 'factor_{}'.format(col)])
        year_1 = (data['mt'].between(1, 12))
        year_2 = (data['mt'].between(13, 24))
        i = 0
        for df in data[col].unique():
            temp1 = data[(data[col] == df) & (year_1)]
            temp2 = data[(data[col] == df) & (year_2)]
            sum1 = temp1['label'].sum()
            sum2 = temp2['label'].sum()
            factor = sum2 / sum1
            temp_df.loc[i] = {col:df, 'factor_{}'.format(col):factor}
            i = i+1
        data = data.merge(temp_df, how='left', on=[col])
    return data

In [5]:
def average_three(data):
    '''构造该月份对应上一年的该月份，该月份的前一个月和该月份的后一个月的平均值，作为新特征'''
    temp_df = pd.DataFrame(columns=['model_adcode', 'mt', 'aver'])
    j = 0
    for i in tqdm([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]):
        for df in data['model_adcode'].unique():
            if (i == 13):
                temp1 = data[(data['model_adcode'] == df) & (data['mt'] == i-12)].reset_index().loc[0]['label']
                temp2 = data[(data['model_adcode'] == df) & (data['mt'] == i-11)].reset_index().loc[0]['label']
                average = (temp1+temp2)/2
                temp_df.loc[j] = {'model_adcode':df, 'mt':i, 'aver':average}
                j = j+1
            elif (i >= 13):
                temp1 = data[(data['model_adcode'] == df) & (data['mt'] == i-12)].reset_index().loc[0]['label']
                temp2 = data[(data['model_adcode'] == df) & (data['mt'] == i-11)].reset_index().loc[0]['label']
                temp3 = data[(data['model_adcode'] == df) & (data['mt'] == i-13)].reset_index().loc[0]['label']
                average = (temp1+temp2+temp3)/3
                temp_df.loc[j] = {'model_adcode':df, 'mt':i, 'aver':average}
                j = j+1
    data = data.merge(temp_df, how='left', on=['model_adcode', 'mt'])
    return data

In [6]:
#计算每种车型销量相关系数
def correlation(data, start, end, mode=0):
    cor = pd.DataFrame()
    for m in tqdm(data['model'].unique()):
        temp = []
        for i in range(1, 24):
            val1 = data[(data['model'] == m) & (data['mt'] == i)]['salesVolume'].sum()
            val2 = data[(data['model'] == m) & (data['mt'] == i+1)]['salesVolume'].sum()
            gap = val2-val1
            temp.append(gap)
        cor[m] = temp
    cor = cor.corr()
    top10 = pd.DataFrame()
    for col in cor.columns:
        temp = cor[col].sort_values(ascending=False)
        index = temp.index.tolist()[1:11]
        top10[col] = index
    '''top10的列分别是model编号，10行代表对应model中的相关系数最高的10种车型'''
    result = pd.DataFrame(columns=['model', 'adcode', 'mt', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10'])
    for i in top10.columns:
        temp = pd.DataFrame(columns=['model', 'adcode', 'mt', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10'])
        for k in data['adcode'].unique():
            for j in range(start,end):
                line = {'model':i, 'mt':j, 'adcode':k,
                       'c1':data[(data['model']==top10.iloc[0,i])&(data['adcode']==k)&(data['mt']==j-1)]['salesVolume'].values[0],
                       'c2':data[(data['model']==top10.iloc[1,i])&(data['adcode']==k)&(data['mt']==j-1)]['salesVolume'].values[0], 
                       'c3':data[(data['model']==top10.iloc[2,i])&(data['adcode']==k)&(data['mt']==j-1)]['salesVolume'].values[0],
                       'c4':data[(data['model']==top10.iloc[3,i])&(data['adcode']==k)&(data['mt']==j-1)]['salesVolume'].values[0],
                       'c5':data[(data['model']==top10.iloc[4,i])&(data['adcode']==k)&(data['mt']==j-1)]['salesVolume'].values[0],
                       'c6':data[(data['model']==top10.iloc[5,i])&(data['adcode']==k)&(data['mt']==j-1)]['salesVolume'].values[0],
                       'c7':data[(data['model']==top10.iloc[6,i])&(data['adcode']==k)&(data['mt']==j-1)]['salesVolume'].values[0],
                       'c8':data[(data['model']==top10.iloc[7,i])&(data['adcode']==k)&(data['mt']==j-1)]['salesVolume'].values[0],
                       'c9':data[(data['model']==top10.iloc[8,i])&(data['adcode']==k)&(data['mt']==j-1)]['salesVolume'].values[0],
                       'c10':data[(data['model']==top10.iloc[9,i])&(data['adcode']==k)&(data['mt']==j-1)]['salesVolume'].values[0]}
                print(line)
                result = result.append(line, ignore_index=True)
    print(result)
    print(data.head())
    if mode == 0:
        data = data.merge(result)
    elif mode == 1:
        data = combine(data, result, start)
    print(data)
    return data
            
        

In [7]:
#data_df, stat_feat = get_stat_feature(data)
#correlation(data_df)
data = correlation(data, 13, 26)

100%|██████████| 60/60 [00:05<00:00, 10.06it/s]


{'model': 0, 'mt': 13, 'adcode': 310000, 'c1': 359.0, 'c2': 374.0, 'c3': 516.0, 'c4': 317.0, 'c5': 473.0, 'c6': 1734.0, 'c7': 1504.0, 'c8': 1293.0, 'c9': 167.0, 'c10': 291.0}
{'model': 0, 'mt': 14, 'adcode': 310000, 'c1': 339.0, 'c2': 264.0, 'c3': 264.0, 'c4': 238.0, 'c5': 415.0, 'c6': 940.0, 'c7': 815.0, 'c8': 851.0, 'c9': 157.0, 'c10': 260.0}
{'model': 0, 'mt': 15, 'adcode': 310000, 'c1': 177.0, 'c2': 140.0, 'c3': 193.0, 'c4': 124.0, 'c5': 199.0, 'c6': 540.0, 'c7': 447.0, 'c8': 433.0, 'c9': 101.0, 'c10': 125.0}
{'model': 0, 'mt': 16, 'adcode': 310000, 'c1': 321.0, 'c2': 173.0, 'c3': 369.0, 'c4': 175.0, 'c5': 338.0, 'c6': 957.0, 'c7': 731.0, 'c8': 764.0, 'c9': 187.0, 'c10': 252.0}
{'model': 0, 'mt': 17, 'adcode': 310000, 'c1': 249.0, 'c2': 111.0, 'c3': 368.0, 'c4': 178.0, 'c5': 303.0, 'c6': 807.0, 'c7': 532.0, 'c8': 667.0, 'c9': 162.0, 'c10': 169.0}
{'model': 0, 'mt': 18, 'adcode': 310000, 'c1': 248.0, 'c2': 133.0, 'c3': 312.0, 'c4': 198.0, 'c5': 272.0, 'c6': 919.0, 'c7': 612.0, 'c8':

{'model': 0, 'mt': 22, 'adcode': 110000, 'c1': 855.0, 'c2': 716.0, 'c3': 493.0, 'c4': 639.0, 'c5': 397.0, 'c6': 472.0, 'c7': 272.0, 'c8': 842.0, 'c9': 412.0, 'c10': 578.0}
{'model': 0, 'mt': 23, 'adcode': 110000, 'c1': 791.0, 'c2': 555.0, 'c3': 438.0, 'c4': 464.0, 'c5': 323.0, 'c6': 373.0, 'c7': 206.0, 'c8': 705.0, 'c9': 404.0, 'c10': 448.0}
{'model': 0, 'mt': 24, 'adcode': 110000, 'c1': 996.0, 'c2': 804.0, 'c3': 628.0, 'c4': 491.0, 'c5': 419.0, 'c6': 574.0, 'c7': 198.0, 'c8': 996.0, 'c9': 331.0, 'c10': 408.0}
{'model': 0, 'mt': 25, 'adcode': 110000, 'c1': 1146.0, 'c2': 747.0, 'c3': 498.0, 'c4': 620.0, 'c5': 494.0, 'c6': 601.0, 'c7': 95.0, 'c8': 839.0, 'c9': 347.0, 'c10': 290.0}
{'model': 0, 'mt': 13, 'adcode': 510000, 'c1': 1083.0, 'c2': 1078.0, 'c3': 543.0, 'c4': 560.0, 'c5': 849.0, 'c6': 1876.0, 'c7': 1775.0, 'c8': 1105.0, 'c9': 434.0, 'c10': 296.0}
{'model': 0, 'mt': 14, 'adcode': 510000, 'c1': 1390.0, 'c2': 940.0, 'c3': 391.0, 'c4': 493.0, 'c5': 933.0, 'c6': 1072.0, 'c7': 1096.0, 

{'model': 0, 'mt': 25, 'adcode': 140000, 'c1': 527.0, 'c2': 427.0, 'c3': 495.0, 'c4': 234.0, 'c5': 173.0, 'c6': 603.0, 'c7': 60.0, 'c8': 772.0, 'c9': 165.0, 'c10': 43.0}
{'model': 0, 'mt': 13, 'adcode': 440000, 'c1': 1243.0, 'c2': 2562.0, 'c3': 3747.0, 'c4': 1893.0, 'c5': 2078.0, 'c6': 2594.0, 'c7': 2216.0, 'c8': 825.0, 'c9': 1829.0, 'c10': 572.0}
{'model': 0, 'mt': 14, 'adcode': 440000, 'c1': 727.0, 'c2': 1849.0, 'c3': 2666.0, 'c4': 1283.0, 'c5': 1526.0, 'c6': 949.0, 'c7': 854.0, 'c8': 423.0, 'c9': 1713.0, 'c10': 437.0}
{'model': 0, 'mt': 15, 'adcode': 440000, 'c1': 322.0, 'c2': 777.0, 'c3': 1125.0, 'c4': 554.0, 'c5': 521.0, 'c6': 463.0, 'c7': 451.0, 'c8': 180.0, 'c9': 1159.0, 'c10': 152.0}
{'model': 0, 'mt': 16, 'adcode': 440000, 'c1': 827.0, 'c2': 1549.0, 'c3': 2377.0, 'c4': 1157.0, 'c5': 1068.0, 'c6': 1024.0, 'c7': 986.0, 'c8': 427.0, 'c9': 1738.0, 'c10': 310.0}
{'model': 0, 'mt': 17, 'adcode': 440000, 'c1': 693.0, 'c2': 1067.0, 'c3': 2460.0, 'c4': 1342.0, 'c5': 875.0, 'c6': 892.0,

{'model': 0, 'mt': 15, 'adcode': 130000, 'c1': 1165.0, 'c2': 407.0, 'c3': 422.0, 'c4': 248.0, 'c5': 431.0, 'c6': 404.0, 'c7': 442.0, 'c8': 884.0, 'c9': 251.0, 'c10': 79.0}
{'model': 0, 'mt': 16, 'adcode': 130000, 'c1': 1697.0, 'c2': 572.0, 'c3': 734.0, 'c4': 489.0, 'c5': 483.0, 'c6': 655.0, 'c7': 599.0, 'c8': 1377.0, 'c9': 371.0, 'c10': 112.0}
{'model': 0, 'mt': 17, 'adcode': 130000, 'c1': 1805.0, 'c2': 721.0, 'c3': 762.0, 'c4': 372.0, 'c5': 470.0, 'c6': 694.0, 'c7': 687.0, 'c8': 1523.0, 'c9': 364.0, 'c10': 113.0}
{'model': 0, 'mt': 18, 'adcode': 130000, 'c1': 2074.0, 'c2': 849.0, 'c3': 790.0, 'c4': 444.0, 'c5': 474.0, 'c6': 773.0, 'c7': 716.0, 'c8': 1631.0, 'c9': 437.0, 'c10': 144.0}
{'model': 0, 'mt': 19, 'adcode': 130000, 'c1': 1714.0, 'c2': 721.0, 'c3': 652.0, 'c4': 470.0, 'c5': 415.0, 'c6': 655.0, 'c7': 543.0, 'c8': 1539.0, 'c9': 425.0, 'c10': 111.0}
{'model': 0, 'mt': 20, 'adcode': 130000, 'c1': 1806.0, 'c2': 962.0, 'c3': 806.0, 'c4': 561.0, 'c5': 495.0, 'c6': 680.0, 'c7': 668.0,

...

{'model': 59, 'mt': 13, 'adcode': 140000, 'c1': 414.0, 'c2': 244.0, 'c3': 197.0, 'c4': 103.0, 'c5': 78.0, 'c6': 1203.0, 'c7': 91.0, 'c8': 260.0, 'c9': 605.0, 'c10': 348.0}
{'model': 59, 'mt': 14, 'adcode': 140000, 'c1': 426.0, 'c2': 213.0, 'c3': 179.0, 'c4': 120.0, 'c5': 105.0, 'c6': 945.0, 'c7': 111.0, 'c8': 248.0, 'c9': 316.0, 'c10': 274.0}
{'model': 59, 'mt': 15, 'adcode': 140000, 'c1': 317.0, 'c2': 113.0, 'c3': 103.0, 'c4': 44.0, 'c5': 64.0, 'c6': 474.0, 'c7': 56.0, 'c8': 146.0, 'c9': 221.0, 'c10': 115.0}
{'model': 59, 'mt': 16, 'adcode': 140000, 'c1': 451.0, 'c2': 141.0, 'c3': 189.0, 'c4': 90.0, 'c5': 83.0, 'c6': 746.0, 'c7': 97.0, 'c8': 249.0, 'c9': 369.0, 'c10': 219.0}
{'model': 59, 'mt': 17, 'adcode': 140000, 'c1': 395.0, 'c2': 101.0, 'c3': 173.0, 'c4': 120.0, 'c5': 78.0, 'c6': 713.0, 'c7': 101.0, 'c8': 182.0, 'c9': 335.0, 'c10': 218.0}
{'model': 59, 'mt': 18, 'adcode': 140000, 'c1': 445.0, 'c2': 103.0, 'c3': 157.0, 'c4': 136.0, 'c5': 62.0, 'c6': 727.0, 'c7': 97.0, 'c8': 251.0,

{'model': 59, 'mt': 22, 'adcode': 320000, 'c1': 1046.0, 'c2': 549.0, 'c3': 476.0, 'c4': 1354.0, 'c5': 353.0, 'c6': 1620.0, 'c7': 797.0, 'c8': 759.0, 'c9': 980.0, 'c10': 728.0}
{'model': 59, 'mt': 23, 'adcode': 320000, 'c1': 1162.0, 'c2': 440.0, 'c3': 350.0, 'c4': 983.0, 'c5': 284.0, 'c6': 1295.0, 'c7': 673.0, 'c8': 664.0, 'c9': 1039.0, 'c10': 718.0}
{'model': 59, 'mt': 24, 'adcode': 320000, 'c1': 1396.0, 'c2': 501.0, 'c3': 444.0, 'c4': 1276.0, 'c5': 342.0, 'c6': 1548.0, 'c7': 806.0, 'c8': 694.0, 'c9': 1098.0, 'c10': 580.0}
{'model': 59, 'mt': 25, 'adcode': 320000, 'c1': 1479.0, 'c2': 550.0, 'c3': 456.0, 'c4': 1308.0, 'c5': 325.0, 'c6': 1918.0, 'c7': 941.0, 'c8': 799.0, 'c9': 1164.0, 'c10': 750.0}
{'model': 59, 'mt': 13, 'adcode': 360000, 'c1': 329.0, 'c2': 262.0, 'c3': 109.0, 'c4': 247.0, 'c5': 37.0, 'c6': 571.0, 'c7': 138.0, 'c8': 262.0, 'c9': 409.0, 'c10': 382.0}
{'model': 59, 'mt': 14, 'adcode': 360000, 'c1': 364.0, 'c2': 297.0, 'c3': 126.0, 'c4': 282.0, 'c5': 61.0, 'c6': 408.0, 'c7

{'model': 59, 'mt': 18, 'adcode': 330000, 'c1': 707.0, 'c2': 311.0, 'c3': 366.0, 'c4': 1017.0, 'c5': 311.0, 'c6': 651.0, 'c7': 720.0, 'c8': 562.0, 'c9': 1108.0, 'c10': 503.0}
{'model': 59, 'mt': 19, 'adcode': 330000, 'c1': 646.0, 'c2': 325.0, 'c3': 380.0, 'c4': 1178.0, 'c5': 449.0, 'c6': 616.0, 'c7': 647.0, 'c8': 591.0, 'c9': 1385.0, 'c10': 544.0}
{'model': 59, 'mt': 20, 'adcode': 330000, 'c1': 663.0, 'c2': 307.0, 'c3': 435.0, 'c4': 1317.0, 'c5': 354.0, 'c6': 628.0, 'c7': 584.0, 'c8': 624.0, 'c9': 1507.0, 'c10': 617.0}
{'model': 59, 'mt': 21, 'adcode': 330000, 'c1': 732.0, 'c2': 322.0, 'c3': 444.0, 'c4': 1460.0, 'c5': 283.0, 'c6': 814.0, 'c7': 732.0, 'c8': 669.0, 'c9': 1239.0, 'c10': 739.0}
{'model': 59, 'mt': 22, 'adcode': 330000, 'c1': 701.0, 'c2': 340.0, 'c3': 478.0, 'c4': 1660.0, 'c5': 365.0, 'c6': 733.0, 'c7': 718.0, 'c8': 649.0, 'c9': 1522.0, 'c10': 738.0}
{'model': 59, 'mt': 23, 'adcode': 330000, 'c1': 734.0, 'c2': 344.0, 'c3': 383.0, 'c4': 1157.0, 'c5': 291.0, 'c6': 614.0, 'c7'

{'model': 59, 'mt': 14, 'adcode': 210000, 'c1': 704.0, 'c2': 249.0, 'c3': 506.0, 'c4': 281.0, 'c5': 158.0, 'c6': 270.0, 'c7': 140.0, 'c8': 650.0, 'c9': 898.0, 'c10': 571.0}
{'model': 59, 'mt': 15, 'adcode': 210000, 'c1': 546.0, 'c2': 117.0, 'c3': 316.0, 'c4': 156.0, 'c5': 103.0, 'c6': 165.0, 'c7': 90.0, 'c8': 566.0, 'c9': 506.0, 'c10': 424.0}
{'model': 59, 'mt': 16, 'adcode': 210000, 'c1': 870.0, 'c2': 274.0, 'c3': 529.0, 'c4': 268.0, 'c5': 124.0, 'c6': 356.0, 'c7': 113.0, 'c8': 775.0, 'c9': 962.0, 'c10': 658.0}
{'model': 59, 'mt': 17, 'adcode': 210000, 'c1': 783.0, 'c2': 320.0, 'c3': 433.0, 'c4': 293.0, 'c5': 129.0, 'c6': 218.0, 'c7': 125.0, 'c8': 692.0, 'c9': 872.0, 'c10': 595.0}
{'model': 59, 'mt': 18, 'adcode': 210000, 'c1': 781.0, 'c2': 298.0, 'c3': 489.0, 'c4': 330.0, 'c5': 115.0, 'c6': 179.0, 'c7': 138.0, 'c8': 738.0, 'c9': 971.0, 'c10': 611.0}
{'model': 59, 'mt': 19, 'adcode': 210000, 'c1': 790.0, 'c2': 258.0, 'c3': 457.0, 'c4': 277.0, 'c5': 148.0, 'c6': 255.0, 'c7': 146.0, 'c8

{'model': 59, 'mt': 23, 'adcode': 230000, 'c1': 813.0, 'c2': 237.0, 'c3': 191.0, 'c4': 95.0, 'c5': 32.0, 'c6': 100.0, 'c7': 56.0, 'c8': 234.0, 'c9': 424.0, 'c10': 378.0}
{'model': 59, 'mt': 24, 'adcode': 230000, 'c1': 1027.0, 'c2': 262.0, 'c3': 233.0, 'c4': 172.0, 'c5': 39.0, 'c6': 74.0, 'c7': 48.0, 'c8': 302.0, 'c9': 571.0, 'c10': 440.0}
{'model': 59, 'mt': 25, 'adcode': 230000, 'c1': 992.0, 'c2': 265.0, 'c3': 316.0, 'c4': 173.0, 'c5': 37.0, 'c6': 91.0, 'c7': 66.0, 'c8': 325.0, 'c9': 573.0, 'c10': 508.0}
       model    adcode    mt      c1     c2     c3     c4     c5      c6  \
0        0.0  310000.0  13.0   359.0  374.0  516.0  317.0  473.0  1734.0   
1        0.0  310000.0  14.0   339.0  264.0  264.0  238.0  415.0   940.0   
2        0.0  310000.0  15.0   177.0  140.0  193.0  124.0  199.0   540.0   
3        0.0  310000.0  16.0   321.0  173.0  369.0  175.0  338.0   957.0   
4        0.0  310000.0  17.0   249.0  111.0  368.0  178.0  303.0   807.0   
...      ...       ...   ...     

In [ ]:
#与correlation函数配套使用，用于将两个表格拼接起来
def combine(data, result, mt):
    

In [8]:
for month in [25,26,27,28]: 
    m_type = 'xgb' 
    
    data = correlation(data, month, month+1)
    
    data_df, stat_feat = get_stat_feature(data)
    
    num_feat = ['regYear'] + stat_feat + ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10']
    cate_feat = ['adcode','bodyType','model','regMonth']
    
    if m_type == 'lgb':
        for i in cate_feat:
            data_df[i] = data_df[i].astype('category')
    elif m_type == 'xgb':
        lbl = LabelEncoder()  
        for i in tqdm(cate_feat):
            data_df[i] = lbl.fit_transform(data_df[i].astype(str))
           
    features = num_feat + cate_feat
    print(len(features), len(set(features)))   
    
    sub,val_pred = get_train_model(data_df, month, m_type)
    #将预测出来的结果再重新加入训练文件，以得到下一个月的结果
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'salesVolume'] = sub['forecastVolum'].values
    data.loc[(data.regMonth==(month-24))&(data.regYear==2018), 'label'      ] = sub['forecastVolum'].values
#ratio = trend_factor(data_df)
#print('ratio is: ' + str(ratio))
sub = data.loc[(data.regMonth>=1)&(data.regYear==2018), ['id','salesVolume']]
print(sub)
sub.columns = ['id','forecastVolum']
#sub['forecastVolum'] = sub['forecastVolum'].apply(lambda x: x * ratio) 
sub[['id','forecastVolum']].round().astype(int).to_csv('../Data/Final/model_2_2.csv', index=False)

100%|██████████| 60/60 [00:05<00:00, 10.49it/s]


{'model': 0, 'mt': 25, 'adcode': 310000, 'c1': 398.0, 'c2': 462.0, 'c3': 554.0, 'c4': 451.0, 'c5': 175.0, 'c6': 338.0, 'c7': 668.0, 'c8': 143.0, 'c9': 616.0, 'c10': 252.0}
{'model': 0, 'mt': 25, 'adcode': 530000, 'c1': 537.0, 'c2': 141.0, 'c3': 267.0, 'c4': 190.0, 'c5': 319.0, 'c6': 334.0, 'c7': 482.0, 'c8': 102.0, 'c9': 250.0, 'c10': 400.0}
{'model': 0, 'mt': 25, 'adcode': 150000, 'c1': 562.0, 'c2': 254.0, 'c3': 106.0, 'c4': 114.0, 'c5': 204.0, 'c6': 181.0, 'c7': 503.0, 'c8': 70.0, 'c9': 88.0, 'c10': 578.0}
{'model': 0, 'mt': 25, 'adcode': 110000, 'c1': 1146.0, 'c2': 498.0, 'c3': 703.0, 'c4': 373.0, 'c5': 347.0, 'c6': 494.0, 'c7': 839.0, 'c8': 75.0, 'c9': 640.0, 'c10': 747.0}
{'model': 0, 'mt': 25, 'adcode': 510000, 'c1': 1191.0, 'c2': 475.0, 'c3': 485.0, 'c4': 421.0, 'c5': 404.0, 'c6': 566.0, 'c7': 880.0, 'c8': 221.0, 'c9': 597.0, 'c10': 896.0}
{'model': 0, 'mt': 25, 'adcode': 340000, 'c1': 754.0, 'c2': 437.0, 'c3': 238.0, 'c4': 251.0, 'c5': 133.0, 'c6': 252.0, 'c7': 1238.0, 'c8': 19

{'model': 2, 'mt': 25, 'adcode': 510000, 'c1': 456.0, 'c2': 485.0, 'c3': 475.0, 'c4': 566.0, 'c5': 40.0, 'c6': 221.0, 'c7': 584.0, 'c8': 284.0, 'c9': 1191.0, 'c10': 896.0}
{'model': 2, 'mt': 25, 'adcode': 340000, 'c1': 204.0, 'c2': 238.0, 'c3': 437.0, 'c4': 252.0, 'c5': 57.0, 'c6': 193.0, 'c7': 348.0, 'c8': 399.0, 'c9': 754.0, 'c10': 302.0}
{'model': 2, 'mt': 25, 'adcode': 370000, 'c1': 471.0, 'c2': 439.0, 'c3': 1080.0, 'c4': 640.0, 'c5': 293.0, 'c6': 165.0, 'c7': 661.0, 'c8': 606.0, 'c9': 2059.0, 'c10': 1000.0}
{'model': 2, 'mt': 25, 'adcode': 140000, 'c1': 200.0, 'c2': 137.0, 'c3': 495.0, 'c4': 173.0, 'c5': 97.0, 'c6': 33.0, 'c7': 160.0, 'c8': 60.0, 'c9': 527.0, 'c10': 427.0}
{'model': 2, 'mt': 25, 'adcode': 440000, 'c1': 2821.0, 'c2': 1060.0, 'c3': 2800.0, 'c4': 1350.0, 'c5': 847.0, 'c6': 677.0, 'c7': 1586.0, 'c8': 454.0, 'c9': 1278.0, 'c10': 2378.0}
{'model': 2, 'mt': 25, 'adcode': 450000, 'c1': 371.0, 'c2': 144.0, 'c3': 152.0, 'c4': 168.0, 'c5': 20.0, 'c6': 190.0, 'c7': 210.0, 'c8

{'model': 4, 'mt': 25, 'adcode': 130000, 'c1': 2104.0, 'c2': 458.0, 'c3': 930.0, 'c4': 1263.0, 'c5': 2875.0, 'c6': 641.0, 'c7': 2644.0, 'c8': 909.0, 'c9': 901.0, 'c10': 268.0}
{'model': 4, 'mt': 25, 'adcode': 410000, 'c1': 5667.0, 'c2': 892.0, 'c3': 527.0, 'c4': 2832.0, 'c5': 2708.0, 'c6': 596.0, 'c7': 2367.0, 'c8': 767.0, 'c9': 777.0, 'c10': 454.0}
{'model': 4, 'mt': 25, 'adcode': 330000, 'c1': 1628.0, 'c2': 1459.0, 'c3': 102.0, 'c4': 1040.0, 'c5': 1439.0, 'c6': 380.0, 'c7': 1312.0, 'c8': 830.0, 'c9': 362.0, 'c10': 94.0}
{'model': 4, 'mt': 25, 'adcode': 420000, 'c1': 719.0, 'c2': 413.0, 'c3': 240.0, 'c4': 558.0, 'c5': 1562.0, 'c6': 352.0, 'c7': 862.0, 'c8': 342.0, 'c9': 76.0, 'c10': 207.0}
{'model': 4, 'mt': 25, 'adcode': 430000, 'c1': 1368.0, 'c2': 361.0, 'c3': 228.0, 'c4': 1420.0, 'c5': 1306.0, 'c6': 249.0, 'c7': 1281.0, 'c8': 304.0, 'c9': 350.0, 'c10': 688.0}
{'model': 4, 'mt': 25, 'adcode': 350000, 'c1': 493.0, 'c2': 275.0, 'c3': 58.0, 'c4': 655.0, 'c5': 628.0, 'c6': 109.0, 'c7': 

{'model': 6, 'mt': 25, 'adcode': 210000, 'c1': 570.0, 'c2': 495.0, 'c3': 194.0, 'c4': 1994.0, 'c5': 795.0, 'c6': 246.0, 'c7': 528.0, 'c8': 1895.0, 'c9': 206.0, 'c10': 127.0}
{'model': 6, 'mt': 25, 'adcode': 500000, 'c1': 150.0, 'c2': 210.0, 'c3': 452.0, 'c4': 349.0, 'c5': 131.0, 'c6': 248.0, 'c7': 653.0, 'c8': 348.0, 'c9': 156.0, 'c10': 21.0}
{'model': 6, 'mt': 25, 'adcode': 610000, 'c1': 274.0, 'c2': 309.0, 'c3': 602.0, 'c4': 547.0, 'c5': 332.0, 'c6': 234.0, 'c7': 545.0, 'c8': 1137.0, 'c9': 158.0, 'c10': 20.0}
{'model': 6, 'mt': 25, 'adcode': 230000, 'c1': 316.0, 'c2': 173.0, 'c3': 91.0, 'c4': 892.0, 'c5': 325.0, 'c6': 86.0, 'c7': 253.0, 'c8': 2042.0, 'c9': 97.0, 'c10': 37.0}
{'model': 7, 'mt': 25, 'adcode': 310000, 'c1': 1335.0, 'c2': 166.0, 'c3': 272.0, 'c4': 157.0, 'c5': 544.0, 'c6': 753.0, 'c7': 106.0, 'c8': 578.0, 'c9': 180.0, 'c10': 134.0}
{'model': 7, 'mt': 25, 'adcode': 530000, 'c1': 451.0, 'c2': 688.0, 'c3': 526.0, 'c4': 201.0, 'c5': 1391.0, 'c6': 449.0, 'c7': 643.0, 'c8': 20

...

{'model': 57, 'mt': 25, 'adcode': 510000, 'c1': 1369.0, 'c2': 1801.0, 'c3': 2246.0, 'c4': 1008.0, 'c5': 1207.0, 'c6': 282.0, 'c7': 320.0, 'c8': 1876.0, 'c9': 589.0, 'c10': 217.0}
{'model': 57, 'mt': 25, 'adcode': 340000, 'c1': 1013.0, 'c2': 1197.0, 'c3': 731.0, 'c4': 1522.0, 'c5': 2004.0, 'c6': 745.0, 'c7': 375.0, 'c8': 1487.0, 'c9': 1540.0, 'c10': 568.0}
{'model': 57, 'mt': 25, 'adcode': 370000, 'c1': 2076.0, 'c2': 3100.0, 'c3': 1563.0, 'c4': 1459.0, 'c5': 1782.0, 'c6': 746.0, 'c7': 664.0, 'c8': 2685.0, 'c9': 3244.0, 'c10': 979.0}
{'model': 57, 'mt': 25, 'adcode': 140000, 'c1': 1389.0, 'c2': 1457.0, 'c3': 1703.0, 'c4': 389.0, 'c5': 537.0, 'c6': 182.0, 'c7': 128.0, 'c8': 1125.0, 'c9': 839.0, 'c10': 378.0}
{'model': 57, 'mt': 25, 'adcode': 440000, 'c1': 2248.0, 'c2': 2825.0, 'c3': 1064.0, 'c4': 1566.0, 'c5': 1686.0, 'c6': 684.0, 'c7': 599.0, 'c8': 1541.0, 'c9': 4687.0, 'c10': 121.0}
{'model': 57, 'mt': 25, 'adcode': 450000, 'c1': 869.0, 'c2': 1161.0, 'c3': 377.0, 'c4': 290.0, 'c5': 660.

{'model': 59, 'mt': 25, 'adcode': 330000, 'c1': 870.0, 'c2': 1059.0, 'c3': 909.0, 'c4': 853.0, 'c5': 1724.0, 'c6': 467.0, 'c7': 656.0, 'c8': 1295.0, 'c9': 433.0, 'c10': 820.0}
{'model': 59, 'mt': 25, 'adcode': 420000, 'c1': 405.0, 'c2': 709.0, 'c3': 378.0, 'c4': 502.0, 'c5': 653.0, 'c6': 291.0, 'c7': 234.0, 'c8': 342.0, 'c9': 238.0, 'c10': 1482.0}
{'model': 59, 'mt': 25, 'adcode': 430000, 'c1': 272.0, 'c2': 648.0, 'c3': 630.0, 'c4': 369.0, 'c5': 628.0, 'c6': 357.0, 'c7': 344.0, 'c8': 301.0, 'c9': 198.0, 'c10': 749.0}
{'model': 59, 'mt': 25, 'adcode': 350000, 'c1': 278.0, 'c2': 518.0, 'c3': 275.0, 'c4': 308.0, 'c5': 487.0, 'c6': 185.0, 'c7': 160.0, 'c8': 365.0, 'c9': 116.0, 'c10': 401.0}
{'model': 59, 'mt': 25, 'adcode': 210000, 'c1': 210.0, 'c2': 881.0, 'c3': 194.0, 'c4': 795.0, 'c5': 1150.0, 'c6': 282.0, 'c7': 141.0, 'c8': 206.0, 'c9': 570.0, 'c10': 1011.0}
{'model': 59, 'mt': 25, 'adcode': 500000, 'c1': 227.0, 'c2': 463.0, 'c3': 452.0, 'c4': 131.0, 'c5': 180.0, 'c6': 445.0, 'c7': 167

100%|██████████| 4/4 [00:00<00:00, 494.51it/s]

      model    adcode    mt      c1     c2     c3     c4      c5     c6  \
0       0.0  310000.0  25.0   398.0  462.0  554.0  451.0   175.0  338.0   
1       0.0  530000.0  25.0   537.0  141.0  267.0  190.0   319.0  334.0   
2       0.0  150000.0  25.0   562.0  254.0  106.0  114.0   204.0  181.0   
3       0.0  110000.0  25.0  1146.0  498.0  703.0  373.0   347.0  494.0   
4       0.0  510000.0  25.0  1191.0  475.0  485.0  421.0   404.0  566.0   
...     ...       ...   ...     ...    ...    ...    ...     ...    ...   
1315   59.0  350000.0  25.0   278.0  518.0  275.0  308.0   487.0  185.0   
1316   59.0  210000.0  25.0   210.0  881.0  194.0  795.0  1150.0  282.0   
1317   59.0  500000.0  25.0   227.0  463.0  452.0  131.0   180.0  445.0   
1318   59.0  610000.0  25.0   173.0  610.0  602.0  332.0   296.0  232.0   
1319   59.0  230000.0  25.0    69.0  992.0   91.0  325.0   573.0  265.0   

         c7     c8     c9     c10  
0     668.0  143.0  616.0   252.0  
1     482.0  102.0  250.0  

XGBoostError: [00:27:08] /home/cauchy_mars/xgboost/src/objective/regression_obj.cu:341: Check failed: info.labels_.Size() != 0U (0 vs. 0) : label set cannot be empty
Stack trace:
  [bt] (0) /usr/local/lib/python3.6/dist-packages/xgboost-1.0.0_SNAPSHOT-py3.6.egg/xgboost/./lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x32) [0x7f2e61a67672]
  [bt] (1) /usr/local/lib/python3.6/dist-packages/xgboost-1.0.0_SNAPSHOT-py3.6.egg/xgboost/./lib/libxgboost.so(xgboost::obj::GammaRegression::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*)+0xf7) [0x7f2e61afbd67]
  [bt] (2) /usr/local/lib/python3.6/dist-packages/xgboost-1.0.0_SNAPSHOT-py3.6.egg/xgboost/./lib/libxgboost.so(xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*)+0x321) [0x7f2e61ac8101]
  [bt] (3) /usr/local/lib/python3.6/dist-packages/xgboost-1.0.0_SNAPSHOT-py3.6.egg/xgboost/./lib/libxgboost.so(XGBoosterUpdate